In [9]:
# coding: utf-8
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline 
import numpy as np
import psycopg2
import pandas as pd
import jieba
import jieba.analyse
import jieba.posseg as pseg
import re
import random
import pickle as pkl
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
jieba.load_userdict("../dataset/dict.txt.big") 

from quantify_judgement import getGuiltyNum

In [2]:
def jieba_tokenize(text):
    return  [t for t in jieba.lcut(text) if re.match("^[\u4e00-\u9fa5]",t)] 

In [3]:
tfidf_vectorizer = pkl.load(open("../dataset/tfidf_vectorizer.pkl" ,'rb'))
# ix2Squid = pkl.load(open("../dataset/ix2Squid.pkl" ,'rb'))

conn = psycopg2.connect(database="law1", user="datac1", password="datac15543", host="ci.lu.im.ntu.edu.tw", port="5432")
print("Opened database successfully")
cur = conn.cursor()

Opened database successfully


In [6]:
testReasonFact = []
label = []
for i in range(11950,11940,-1):
    cur.execute("""SELECT sqlid,main,reasonfact FROM jdata_features WHERE sqlid IN (SELECT sqlid FROM jdata_meta WHERE cat = 'M' and parser_done = 1 and sqlid >= %d and sqlid < %d) """ % ((i-1)*1000,i*1000))
    rows = cur.fetchall()
    for row in rows:
        #print(row)
        guiltyNum = getGuiltyNum(row[0],conn)
        if not (np.isnan(guiltyNum)) :
            testReasonFact.append(row[1])
            label.append(int(guiltyNum))
        #sqlidRsts.append([row[0],random.randint(0,1200)])
    print(i,len(testReasonFact))

11950 302
11949 597
11948 657
11947 701
11946 823
11945 828
11944 828
11943 1029
11942 1163
11941 1481


In [7]:
tfidf_test_matrix = tfidf_vectorizer.transform(testReasonFact)

print(tfidf_test_matrix.shape)

#print(sqlidRsts)

(1481, 7270830)


In [57]:
clf = TruncatedSVD(n_components=150)

#pca = PCA(n_components=2)
lsa_matrix = clf.fit_transform(tfidf_test_matrix)

#print(pca_matrix[:,0])

In [12]:
print(lsa_matrix[0])

[  2.56550391e-01  -9.71061308e-02   9.56003707e-02  -1.42473483e-01
   1.02396705e-01  -6.06848935e-02   3.30855740e-01   9.87520700e-02
  -3.18890051e-01   8.42926647e-02   1.37724107e-01   4.74744949e-02
  -2.97565239e-01   1.71557803e-01   1.92199432e-02  -9.10885722e-02
  -1.65945384e-02   3.27687171e-02  -8.58823645e-02  -8.96757611e-02
   1.36434936e-01  -2.38809502e-02  -1.70408128e-02   4.33725095e-02
  -6.74596741e-02  -7.01528289e-05  -5.23051517e-02   2.55261748e-02
   5.55240796e-02  -4.64488818e-03  -1.01626661e-01  -2.41283633e-02
  -3.74460734e-02   1.95499375e-02   3.05331894e-02   2.68197389e-02
   1.28101133e-02  -2.87788499e-02   1.25499259e-02  -5.53928288e-03
  -1.75174321e-02  -2.01244222e-02   2.43486498e-02  -6.03392561e-02
  -9.28925169e-03   3.98539551e-02   3.41220937e-02   6.00458484e-02
  -1.64828686e-02   8.83023250e-03   1.59678637e-02   4.88390920e-03
  -2.66017880e-03  -3.06179155e-03   1.80615875e-02  -6.94512712e-03
   1.35047405e-02   1.63156929e-03

In [15]:
def split_data(X,Y,split_ratio):
    indices = np.arange(X.shape[0])  
    np.random.shuffle(indices) 
    X = np.array(X)
    Y = np.array(Y)

    X_data = X[indices]
    Y_data = Y[indices]

    num_validation_sample = int(split_ratio * X_data.shape[0] )

    X_train = X_data[num_validation_sample:]
    Y_train = Y_data[num_validation_sample:]

    X_val = X_data[:num_validation_sample]
    Y_val = Y_data[:num_validation_sample]

    return (X_train,Y_train),(X_val,Y_val)

In [16]:
(X_train,Y_train),(X_val,Y_val) = split_data(lsa_matrix,label,0.2)

In [54]:
#sqlidRsts = [[t[0],t[1]] for t in rawData]
#for k in range(3,13):
#    print("k = %d" % k)
    
#     fig = plt.figure()
    
#     title = "Scatter for k=%d" % k
#     colors =  plt.cm.tab20( (20/k * np.arange(k)).astype(int) )
    
#     patch = []
#     for z in range(0,k):
#         patch.append(mpatches.Patch(color=colors[z], label = z))
#     plt.legend(handles=patch)
#     plt.title(title)

rf = RandomForestClassifier(n_estimators = 450,verbose = 1)

rf.fit(X_train,Y_train)

print(rf.feature_importances_)
    
#    result = rf.fit_predict(tfidf_test_matrix)
    
#     plt.scatter(pca_matrix[:,0],pca_matrix[:,1],c=colors)
#     #plt.show()
#     fig.savefig("img/PCA_%d.png" % k,bbox_inches='tight')

#    for i in range(0,len(result)):
#        sqlidRsts[i].append(result[i])

[ 0.01158512  0.03930435  0.01722949  0.01171481  0.0164589   0.01608477
  0.01264286  0.01906149  0.03880261  0.03061503  0.02939208  0.05543054
  0.01355092  0.02822788  0.03139955  0.03463236  0.03326799  0.0105767
  0.01329896  0.01688349  0.01114432  0.01600236  0.01614369  0.01154288
  0.00683996  0.00786782  0.0067794   0.01250553  0.00810622  0.0071101
  0.00750367  0.00705169  0.00713422  0.00673237  0.00701278  0.00768039
  0.00625187  0.00731801  0.0070732   0.00633422  0.00643684  0.00954531
  0.00651777  0.0056395   0.00677042  0.00559434  0.00639974  0.00587687
  0.00589906  0.00577425  0.00560952  0.0056051   0.00478006  0.0052889
  0.0050777   0.0062676   0.00620703  0.00453973  0.0049676   0.0050184
  0.00621599  0.00577487  0.0051954   0.00557126  0.00535315  0.00579752
  0.00496058  0.00662115  0.0057934   0.00526402  0.00633675  0.00525575
  0.00590011  0.00554658  0.00510218  0.00572187  0.00527392  0.00586826
  0.00504674  0.0057485   0.00576271  0.00547441  0.005

[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed:    7.9s finished


In [55]:
def RMSE(pred,est):
    se = 0.0
    for i in range(0,len(pred)):
        se = se + (pred[i]-est[i]) * (pred[i]-est[i])
    return np.sqrt(se/len(pred))

In [56]:
rst = rf.predict(X_val)
print(RMSE(rst,Y_val))


38.4094670903


[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed:    0.1s finished
